In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.widgets import Slider, Button, RadioButtons
import pandas as pd

# Reservoirs with discharge data
reservoirs = [
    'castaic',
    'del_valle',
    'feather',
    'pyramid',
    'san_luis',
    'thermalito']

noaa_sites = {
    'castaic': 'USC00048014',
    'del_valle': 'USC00045933',
    'feather': 'USC00047195',
    'pyramid': 'USW00023187',
    'san_luis': 'USC00045119',
    'thermalito': 'USC00046521',
}

def load_noaa_data(reservoir):
    if not reservoir in reservoirs:
        print('Reservoir not found')
        return
    
    filename = '../data/'+reservoir+'/noaa_'+noaa_sites[reservoir]+".csv"
    
    return pd.read_csv(filename,index_col="DATE",low_memory=False)

default_reservoir = 'feather'

noaa_data = load_noaa_data(default_reservoir)
dates = pd.to_datetime(noaa_data.index.get_values())
precipitation = noaa_data.loc[:,"PRCP"]

min_date = dates[0]
max_date = dates[-1]

fig = plt.figure(figsize=(10, 6), dpi=80)
ax = fig.gca()

# Shrink the plot by 25% towards upper-right corner
plt.subplots_adjust(left=0.25, bottom=0.25)
l, = plt.plot(dates, precipitation.values, linewidth=2, color='red')

plt.xlabel('Date')
plt.ylabel('Watershed Precipitation (in)')
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
ax.set_title(default_reservoir)

axcolor = 'lightgoldenrodyellow'
axyear_start = plt.axes([0.25, 0.125, 0.65, 0.03], facecolor=axcolor)
axyear_end = plt.axes([0.25, 0.085, 0.65, 0.03], facecolor=axcolor)

syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')

def find_index(dates, year):
    index = 0
    for date in dates:
        if date.year == int(year):
            return index
        index += 1
    return -1

def update(val):
    year_start = syear_start.val
    year_end = syear_end.val
    if not year_start < year_end:
        return
    s_index = find_index(dates, year_start)
    e_index = find_index(dates, year_end + 1)
    l.set_xdata(dates[s_index:e_index])
    l.set_ydata(precipitation[s_index:e_index])
    ax.relim()
    ax.autoscale_view()
    plt.draw()
syear_start.on_changed(update)
syear_end.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

def reset(event):
    global syear_start, syear_end, reservoir_radio, min_date, max_date, dates, precipitation
    reservoir = default_reservoir
    # Load new data
    noaa_data = load_noaa_data(reservoir)
    dates = pd.to_datetime(noaa_data.index.get_values())
    precipitation = noaa_data.loc[:,"PRCP"]
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(precipitation)
    ax.set_title(reservoir)
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset radio buttons
    rax.clear()
    reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))
    reservoir_radio.on_clicked(set_reservoir)
    fig.canvas.flush_events()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.6, 0.15, 0.3], facecolor=axcolor)
reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))

def set_reservoir(label):
    global syear_start, syear_end, station_radio, min_date, max_date, dates, precipitation
    reservoir = label
    # Load new data
    noaa_data = load_noaa_data(reservoir)
    dates = pd.to_datetime(noaa_data.index.get_values())
    precipitation = noaa_data.loc[:,"PRCP"]
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(precipitation)
    ax.set_title(reservoir)
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    fig.canvas.flush_events()
reservoir_radio.on_clicked(set_reservoir)

plt.show()

FigureCanvasNbAgg()

## Summary statistics

The following code prints the summary statistics for each reservoir.

In [3]:
for reservoir in reservoirs:
    noaa_data = load_noaa_data(reservoir)
    precipitation = noaa_data.loc[:,"PRCP"]
    print(reservoir)
    print(precipitation.describe())
    print('\n')

castaic
count    33786.000000
mean         0.050907
std          0.248140
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          5.800000
Name: PRCP, dtype: float64


del_valle
count    24955.000000
mean         0.064905
std          0.230147
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          6.870000
Name: PRCP, dtype: float64


feather
count    40335.000000
mean         0.113501
std          0.392442
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          9.000000
Name: PRCP, dtype: float64


pyramid
count    25277.000000
mean         0.031612
std          0.199636
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          6.320000
Name: PRCP, dtype: float64


san_luis
count    27799.000000
mean         0.023741
std          0.104985
min          0.000000
25%          0.000000
50%          0.000000
75%  